In [2]:
try:
    import torch_geometric
except ImportError:
    !pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.8 MB/s eta 0:00:00


In [3]:
try:
    import implicit
    import lightfm
except ImportError:
    !pip install -qU implicit
    !pip install lightfm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 97.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 17.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=831129 sha256=d44ea35591938ec4790b73fd58f8c778e89f77a920ff93773d81e68e60d52af3
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [4]:
import json
import torch
import torch.nn.functional as F
import torch.nn as nn
import networkx as nx
import numpy as np
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from torch_geometric.nn import GATConv
from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ndcg_score

In [5]:
import pandas as pd
import numpy as np
from sklearn.metrics import precision_score, recall_score, ndcg_score, accuracy_score
from sklearn.decomposition import TruncatedSVD, PCA, NMF
from sklearn.neighbors import NearestNeighbors
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import csr_matrix, coo_matrix
from implicit.als import AlternatingLeastSquares
import warnings
import matplotlib.pyplot as plt

12.06 Новые изменения
В новой версии системы добавил пользовательское разделение выборки, чтобы тестовая выборка включала только новые сервисы, что соответствует требованию оценивать рекомендации для ранее не использованных сервисов.

Также улучшил извлечение путей в DAG с помощью nx.all_simple_paths, обеспечив более точное контекстное моделирование последовательностей сервисов.

 Эти изменения привели к значительному улучшению nDCG для моделей NMF (+0.0524) и SVD (+0.0682), а также для их гибридных версий.

In [7]:
import pandas as pd
import numpy as np
import torch
import torch.nn.functional as F
import networkx as nx
import json
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ndcg_score
from sklearn.decomposition import TruncatedSVD, NMF
from lightfm import LightFM
from lightfm.data import Dataset
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import NearestNeighbors
import torch.nn as nn
from torch_geometric.nn import GATConv

# ==================== Data Loading and Preprocessing ====================
df = pd.read_csv("calls.csv", sep=";")
df = df[~df['owner'].str.contains('cookies', na=False)]
df = df.sort_values('start_time')

owners = df['owner'].unique()
mids = df['mid'].unique()

# Create user-specific train-test splits
def create_user_splits(df, owners, train_ratio=0.7):
    train_dfs = []
    test_dfs = []
    for owner in owners:
        user_df = df[df['owner'] == owner].sort_values('start_time')
        train_size = int(len(user_df) * train_ratio)
        train_df = user_df[:train_size]
        test_df = user_df[train_size:]

        # Identify services used in training
        used_services = set(train_df['mid'].unique())
        # Filter test set to include only sequences ending with a new service
        test_df = test_df[test_df['mid'].apply(lambda x: x not in used_services)]
        if not test_df.empty:
            train_dfs.append(train_df)
            test_dfs.append(test_df)

    return pd.concat(train_dfs), pd.concat(test_dfs)

df_train, df_test = create_user_splits(df, owners)

def build_matrix(df, owners, mids):
    pivot = df.pivot_table(index='owner', columns='mid', values='id', aggfunc='count').fillna(0)
    pivot = pivot.reindex(index=owners, columns=mids, fill_value=0)
    return pivot.values

X_train = build_matrix(df_train, owners, mids)
X_test = build_matrix(df_test, owners, mids)

with open("compositionsDAG.json", "r", encoding="utf-8") as f:
    data = json.load(f)

dag = nx.DiGraph()
id_to_mid = {}
for composition in data:
    for node in composition["nodes"]:
        if "mid" in node:
            id_to_mid[str(node["id"])] = f"service_{node['mid']}"
        else:
            id_to_mid[str(node["id"])] = f"table_{node['id']}"
    for link in composition["links"]:
        source = str(link["source"])
        target = str(link["target"])
        src_node = id_to_mid[source]
        tgt_node = id_to_mid[target]
        dag.add_node(src_node, type='service' if src_node.startswith("service") else 'table')
        dag.add_node(tgt_node, type='service' if tgt_node.startswith("service") else 'table')
        dag.add_edge(src_node, tgt_node)

# ==================== Model Definitions ====================
class SVDRecommender:
    def __init__(self, model):
        self.model = model

    def fit(self, X):
        self.X = X
        self.embeddings = self.model.fit_transform(X)

    def predict(self):
        return self.model.inverse_transform(self.embeddings)

class KNNRecommender:
    def __init__(self, n_neighbors=4):
        self.model = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
        self.n_neighbors = n_neighbors

    def fit(self, X):
        self.X = X
        self.model.fit(X)

    def predict(self):
        distances, indices = self.model.kneighbors(self.X, n_neighbors=self.n_neighbors)
        preds = np.zeros_like(self.X)
        for i in range(len(self.X)):
            neighbor_scores = self.X[indices[i]].mean(axis=0)
            preds[i] = neighbor_scores
        return preds

class LightFMRecommender:
    def __init__(self, loss='warp'):
        self.loss = loss

    def fit(self, df, owners, mids):
        dataset = Dataset()
        dataset.fit(owners, mids)
        interactions, _ = dataset.build_interactions([(row['owner'], row['mid']) for _, row in df.iterrows()])
        self.model = LightFM(loss=self.loss)
        self.model.fit(interactions, epochs=10, num_threads=4)
        self.dataset = dataset
        self.owners = owners
        self.mids = mids

    def predict(self):
        uid_map = self.dataset.mapping()[0]
        iid_map = self.dataset.mapping()[2]
        num_items = len(iid_map)
        preds = []
        for user in self.owners:
            uid = uid_map[user]
            scores = self.model.predict(uid, np.arange(num_items))
            preds.append(scores / (np.max(np.abs(scores)) + 1e-10))  # Normalize scores
        return np.array(preds)

class ALSRecommender:
    def fit(self, X):
        self.model = AlternatingLeastSquares(factors=20, iterations=10)
        self.X = csr_matrix(X * 20)
        self.model.fit(self.X)

    def predict(self):
        return self.model.user_factors @ self.model.item_factors.T

class WRMFRecommender:
    def fit(self, X):
        self.model = AlternatingLeastSquares(factors=20, iterations=10)
        self.X = csr_matrix(X * 20)
        self.model.fit(self.X)

    def predict(self):
        return self.model.user_factors @ self.model.item_factors.T

class GCNRecommender(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

class HybridRecommender:
    def __init__(self, base_model, gcn_hidden=16, alpha=0.5, is_lightfm=False):
        self.base_model = base_model
        self.gcn = GCNRecommender(in_channels=2, hidden_channels=gcn_hidden, out_channels=len(mids))
        self.alpha = alpha
        self.is_lightfm = is_lightfm
        self.mid_to_idx = {mid: idx for idx, mid in enumerate(mids)}

    def fit(self, X_train, df_train, dag):
        if self.is_lightfm:
            self.base_model.fit(df_train, owners, mids)
        else:
            self.base_model.fit(X_train)

        node_list = list(dag.nodes)
        node_encoder = LabelEncoder()
        node_ids = node_encoder.fit_transform(node_list)
        node_map = {node: idx for node, idx in zip(node_list, node_ids)}
        edge_index = torch.tensor([[node_map[u], node_map[v]] for u, v in dag.edges], dtype=torch.long).t()
        features = [[1, 0] if dag.nodes[n]['type'] == 'service' else [0, 1] for n in node_list]
        x = torch.tensor(features, dtype=torch.float)
        self.data_pyg = Data(x=x, edge_index=edge_index)
        self.node_map = node_map

        paths = []
        service_nodes = [n for n in dag.nodes if n.startswith("service")]
        for start_node in dag.nodes:
            if dag.out_degree(start_node) > 0:
                for target_node in service_nodes:
                    if nx.has_path(dag, start_node, target_node):
                        for path in nx.all_simple_paths(dag, source=start_node, target=target_node, cutoff=5):
                            if len(path) > 2 and path[-1].startswith("service"):
                                paths.append(path)

        contexts = []
        targets = []
        for path in paths:
            for i in range(1, len(path) - 1):
                context = path[i-1]
                next_step = path[i]
                if next_step.startswith("service"):
                    mid = int(next_step.split("_")[1])
                    if mid in self.mid_to_idx:
                        contexts.append(node_map[context])
                        targets.append(self.mid_to_idx[mid])

        self.contexts = torch.tensor(contexts, dtype=torch.long) if contexts else torch.tensor([], dtype=torch.long)
        self.targets = torch.tensor(targets, dtype=torch.long) if targets else torch.tensor([], dtype=torch.long)

        opt = torch.optim.Adam(self.gcn.parameters(), lr=0.01)
        self.gcn.train()
        for epoch in range(300):
            opt.zero_grad()
            if self.contexts.numel() > 0:
                out = self.gcn(self.data_pyg)[self.contexts]
                loss = F.cross_entropy(out, self.targets)
                loss.backward()
                opt.step()

    def predict(self):
        preds_base = self.base_model.predict()
        self.gcn.eval()
        with torch.no_grad():
            gcn_output = self.gcn(self.data_pyg)[self.contexts] if self.contexts.numel() > 0 else torch.zeros((0, len(self.mid_to_idx)))
            gcn_proba = torch.nn.functional.softmax(gcn_output, dim=1).numpy() if gcn_output.numel() > 0 else np.zeros((0, len(self.mid_to_idx)))

        preds_hybrid = np.zeros_like(preds_base)
        for i in range(len(preds_base)):
            gcn_scores = np.mean(gcn_proba, axis=0) if gcn_proba.size > 0 else np.zeros(len(self.mid_to_idx))
            preds_hybrid[i] = self.alpha * preds_base[i] + (1 - self.alpha) * gcn_scores
        return preds_hybrid

# ==================== Utility Functions ====================
def get_used_services(X):
    return [set(np.where(row > 0)[0]) for row in X]

def get_recommendations(scores, used, popular, k):
    recs = []
    for i, user_scores in enumerate(scores):
        sorted_items = np.argsort(user_scores)[::-1]
        filtered = [item for item in sorted_items if item not in used[i]]
        if len(filtered) < k:
            extra = [item for item in popular if item not in used[i] and item not in filtered]
            filtered += extra
        recs.append(filtered[:k])
    return recs

def evaluate(recs, actual, all_items, scores, k):
    results = {'accuracy': [], 'precision': [], 'recall': [], 'ndcg': []}
    for i in range(len(recs)):
        y_true = [1 if j in actual[i] else 0 for j in range(len(all_items))]
        y_pred = [1 if j in recs[i] else 0 for j in range(len(all_items))]
        rel_true = [1 if j in actual[i] else 0 for j in recs[i]]
        rel_score = [scores[i][j] for j in recs[i]]

        results['accuracy'].append(accuracy_score(y_true, y_pred))
        results['precision'].append(precision_score(y_true, y_pred, zero_division=0))
        results['recall'].append(recall_score(y_true, y_pred, zero_division=0))

        true_bin = np.zeros(len(all_items))
        for j in actual[i]:
            true_bin[j] = 1
        score_bin = np.zeros(len(all_items))
        for j, idx in enumerate(recs[i]):
            score_bin[idx] = rel_score[j]
        try:
            results['ndcg'].append(ndcg_score([true_bin], [score_bin], k=k) if sum(true_bin) > 0 else 0)
        except Exception as e:
            print(f"nDCG error for user {i}: {e}")
            results['ndcg'].append(0)

    return {m: np.mean(vals) for m, vals in results.items()}

# ==================== Model Definitions for Composition ====================
class DAGNN(nn.Module):
    def __init__(self, in_channels: int, K: int):
        super().__init__()
        from torch_geometric.nn import APPNP
        self.propagation = APPNP(K=K, alpha=0)
        self.att = nn.Parameter(torch.Tensor(K + 1))
        self.reset_parameters()

    def reset_parameters(self):
        self.propagation.reset_parameters()
        nn.init.zeros_(self.att)

    def forward(self, x, edge_index):
        xs = [x]
        edge_weight = torch.ones(edge_index.size(1), dtype=torch.float32, device=edge_index.device)
        for _ in range(self.propagation.K):
            x = self.propagation.propagate(edge_index, x=x, edge_weight=edge_weight)
            xs.append(x)
        out = torch.stack(xs, dim=-1)
        out = (out * self.att.view(1, 1, -1)).sum(dim=-1)
        return out

class DAGNNRecommender(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
        self.lin = nn.Linear(out_channels, out_channels)

    def forward(self, x, edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.conv2(x, edge_index)
        x = self.lin(x)
        return x

class GATRecommender(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=2):
        super().__init__()
        self.gat1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.gat2 = GATConv(hidden_channels * heads, out_channels, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.elu(self.gat1(x, edge_index))
        x = self.gat2(x, edge_index)
        return x


# ==================== Model Evaluation ====================
base_models = {
    'KNN': KNNRecommender(n_neighbors=4),
    'NMF': SVDRecommender(NMF(n_components=5, init='random', random_state=0)),
    'SVD': SVDRecommender(TruncatedSVD(n_components=5)),
    'LightFM-WARP': LightFMRecommender(loss='warp'),
    'PHCF-BPR': LightFMRecommender(loss='bpr'),
    'ALS': ALSRecommender(),
    'WRMF': WRMFRecommender()
}

hybrid_models = {
    'KNN+GCN': HybridRecommender(base_models['KNN'], gcn_hidden=16, alpha=0.5),
    'NMF+GCN': HybridRecommender(base_models['NMF'], gcn_hidden=16, alpha=0.5),
    'SVD+GCN': HybridRecommender(base_models['SVD'], gcn_hidden=16, alpha=0.5),
    'LightFM-WARP+GCN': HybridRecommender(base_models['LightFM-WARP'], gcn_hidden=16, alpha=0.5, is_lightfm=True),
    'PHCF-BPR+GCN': HybridRecommender(base_models['PHCF-BPR'], gcn_hidden=16, alpha=0.5, is_lightfm=True),
    'ALS+GCN': HybridRecommender(base_models['ALS'], gcn_hidden=16, alpha=0.5),
    'WRMF+GCN': HybridRecommender(base_models['WRMF'], gcn_hidden=16, alpha=0.5)
}

all_models = {**base_models, **hybrid_models,
              'DAGNN': DAGNNRecommender(in_channels=2, hidden_channels=16, out_channels=len(mids)),
              'GAT': GATRecommender(in_channels=2, hidden_channels=16, out_channels=len(mids), heads=2),
              'GCN': GCNRecommender(in_channels=2, hidden_channels=16, out_channels=len(mids))}

popular_services = np.argsort(df_train['mid'].value_counts().reindex(mids, fill_value=0).values)[::-1]
used_train = get_used_services(X_train)
actual_test = get_used_services(X_test)
k_values = [5, 10, 15]
alpha_values = [0.1, 0.3, 0.5, 0.7, 0.9]

results_base = {model: {k: {} for k in k_values} for model in base_models.keys()}
results_hybrid = {model: {k: {} for k in k_values} for model in hybrid_models.keys()}
results_hybrid_alpha = {model: {alpha: {k: {} for k in k_values} for alpha in alpha_values} for model in hybrid_models.keys()}
best_alpha_results = {model: {'alpha': None, 'ndcg': 0, 'metrics': {}} for model in hybrid_models.keys()}

print("\n=== Base Models ===")
for name, model in base_models.items():
    print(f"\n=== {name} ===")
    if name in ['LightFM-WARP', 'PHCF-BPR']:
        model.fit(df_train, owners, mids)
        preds = model.predict()
    else:
        model.fit(X_train)
        preds = model.predict()

    for k in k_values:
        recs = get_recommendations(preds, used_train, popular_services, k)
        metrics = evaluate(recs, actual_test, mids, preds, k)
        results_base[name][k] = metrics
        print(f"\nk={k}:")
        for m, v in metrics.items():
            print(f"{m.capitalize()}: {v:.4f}")

print("\n=== Hybrid Models with Alpha Tuning ===")
for name, model in hybrid_models.items():
    print(f"\n=== {name} ===")
    for alpha in alpha_values:
        print(f"\nTesting alpha={alpha}")
        model.alpha = alpha
        model.fit(X_train, df_train, dag)
        preds = model.predict()

        for k in k_values:
            recs = get_recommendations(preds, used_train, popular_services, k)
            metrics = evaluate(recs, actual_test, mids, preds, k)
            results_hybrid_alpha[name][alpha][k] = metrics
            print(f"\nk={k} (alpha={alpha}):")
            for m, v in metrics.items():
                print(f"{m.capitalize()}: {v:.4f}")

            if k == 5 and metrics['ndcg'] > best_alpha_results[name]['ndcg']:
                best_alpha_results[name] = {'alpha': alpha, 'ndcg': metrics['ndcg'], 'metrics': metrics}

    best_alpha = best_alpha_results[name]['alpha']
    print(f"\nBest alpha for {name}: {best_alpha}")
    model.alpha = best_alpha
    model.fit(X_train, df_train, dag)
    preds = model.predict()

    for k in k_values:
        recs = get_recommendations(preds, used_train, popular_services, k)
        metrics = evaluate(recs, actual_test, mids, preds, k)
        results_hybrid[name][k] = metrics
        print(f"\nk={k} (best alpha={best_alpha}):")
        for m, v in metrics.items():
            print(f"{m.capitalize()}: {v:.4f}")

# ==================== Metric Tables ====================
def create_metric_table(results, models, k_values, title, best_alpha_results=None):
    data = []
    for model in models:
        for k in k_values:
            metrics = results[model][k]
            alpha_str = f" (alpha={best_alpha_results[model]['alpha']})" if best_alpha_results and model in best_alpha_results else ""
            data.append({
                'Model': model + alpha_str,
                'k': k,
                'Accuracy': metrics['accuracy'],
                'Precision': metrics['precision'],
                'Recall': metrics['recall'],
                'nDCG': metrics['ndcg']
            })
    df_table = pd.DataFrame(data)
    df_table = df_table.round(4)
    print(f"\n{title}")
    print(df_table.to_markdown(index=False))
    return df_table

def create_alpha_table(results_alpha, models, alpha_values, k_values):
    data = []
    for model in models:
        for alpha in alpha_values:
            for k in k_values:
                metrics = results_alpha[model][alpha][k]
                data.append({
                    'Model': model,
                    'Alpha': alpha,
                    'k': k,
                    'Accuracy': metrics['accuracy'],
                    'Precision': metrics['precision'],
                    'Recall': metrics['recall'],
                    'nDCG': metrics['ndcg']
                })
    df_table = pd.DataFrame(data)
    df_table = df_table.round(4)
    print("\nAlpha Tuning Results for Hybrid Models")
    print(df_table.to_markdown(index=False))
    return df_table

base_table = create_metric_table(results_base, base_models.keys(), k_values, "Base Models Metrics")
hybrid_table = create_metric_table(results_hybrid, hybrid_models.keys(), k_values, "Hybrid Models Metrics (Best Alpha)", best_alpha_results)
alpha_table = create_alpha_table(results_hybrid_alpha, hybrid_models.keys(), alpha_values, k_values)


=== Base Models ===

=== KNN ===

k=5:
Accuracy: 0.9453
Precision: 0.0737
Recall: 0.0679
Ndcg: 0.0819

k=10:
Accuracy: 0.9217
Precision: 0.0526
Recall: 0.1095
Ndcg: 0.0814

k=15:
Accuracy: 0.8982
Precision: 0.0456
Recall: 0.1802
Ndcg: 0.0978

=== NMF ===

k=5:
Accuracy: 0.9463
Precision: 0.0947
Recall: 0.0859
Ndcg: 0.1324

k=10:
Accuracy: 0.9228
Precision: 0.0632
Recall: 0.0935
Ndcg: 0.1221

k=15:
Accuracy: 0.8982
Precision: 0.0456
Recall: 0.0940
Ndcg: 0.1154

=== SVD ===

k=5:
Accuracy: 0.9453
Precision: 0.0737
Recall: 0.0788
Ndcg: 0.1159

k=10:
Accuracy: 0.9217
Precision: 0.0526
Recall: 0.0864
Ndcg: 0.1094

k=15:
Accuracy: 0.8982
Precision: 0.0456
Recall: 0.0940
Ndcg: 0.1091

=== LightFM-WARP ===

k=5:
Accuracy: 0.9442
Precision: 0.0526
Recall: 0.0547
Ndcg: 0.0541

k=10:
Accuracy: 0.9217
Precision: 0.0526
Recall: 0.0804
Ndcg: 0.0568

k=15:
Accuracy: 0.8992
Precision: 0.0526
Recall: 0.1231
Ndcg: 0.0586

=== PHCF-BPR ===

k=5:
Accuracy: 0.9453
Precision: 0.0737
Recall: 0.0679
Ndcg: 0.

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]


k=5:
Accuracy: 0.9421
Precision: 0.0105
Recall: 0.0066
Ndcg: 0.0179

k=10:
Accuracy: 0.9185
Precision: 0.0211
Recall: 0.0252
Ndcg: 0.0295

k=15:
Accuracy: 0.8939
Precision: 0.0175
Recall: 0.0257
Ndcg: 0.0318

=== WRMF ===


  0%|          | 0/10 [00:00<?, ?it/s]

...


k=5 (alpha=0.7):
Accuracy: 0.9431
Precision: 0.0316
Recall: 0.0076
Ndcg: 0.0470

k=10 (alpha=0.7):
Accuracy: 0.9185
Precision: 0.0211
Recall: 0.0081
Ndcg: 0.0356

k=15 (alpha=0.7):
Accuracy: 0.8966
Precision: 0.0351
Recall: 0.0804
Ndcg: 0.0602

Testing alpha=0.9


  0%|          | 0/10 [00:00<?, ?it/s]


k=5 (alpha=0.9):
Accuracy: 0.9426
Precision: 0.0211
Recall: 0.0071
Ndcg: 0.0357

k=10 (alpha=0.9):
Accuracy: 0.9185
Precision: 0.0211
Recall: 0.0252
Ndcg: 0.0360

k=15 (alpha=0.9):
Accuracy: 0.8945
Precision: 0.0211
Recall: 0.0262
Ndcg: 0.0375

Best alpha for WRMF+GCN: 0.5


  0%|          | 0/10 [00:00<?, ?it/s]


k=5 (best alpha=0.5):
Accuracy: 0.9442
Precision: 0.0526
Recall: 0.0086
Ndcg: 0.0636

k=10 (best alpha=0.5):
Accuracy: 0.9196
Precision: 0.0316
Recall: 0.0262
Ndcg: 0.0501

k=15 (best alpha=0.5):
Accuracy: 0.8955
Precision: 0.0281
Recall: 0.0272
Ndcg: 0.0482

Base Models Metrics
| Model        |   k |   Accuracy |   Precision |   Recall |   nDCG |
|:-------------|----:|-----------:|------------:|---------:|-------:|
| KNN          |   5 |     0.9453 |      0.0737 |   0.0679 | 0.0819 |
| KNN          |  10 |     0.9217 |      0.0526 |   0.1095 | 0.0814 |
| KNN          |  15 |     0.8982 |      0.0456 |   0.1802 | 0.0978 |
| NMF          |   5 |     0.9463 |      0.0947 |   0.0859 | 0.1324 |
| NMF          |  10 |     0.9228 |      0.0632 |   0.0935 | 0.1221 |
| NMF          |  15 |     0.8982 |      0.0456 |   0.094  | 0.1154 |
| SVD          |   5 |     0.9453 |      0.0737 |   0.0788 | 0.1159 |
| SVD          |  10 |     0.9217 |      0.0526 |   0.0864 | 0.1094 |
| SVD          |  1

In [8]:
class GCNRecommender(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super().__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [16]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from sklearn.preprocessing import LabelEncoder, LabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ndcg_score
import networkx as nx
import numpy as np
import pandas as pd
import random
from tqdm import tqdm

# Установка seed для воспроизводимости
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed) if torch.cuda.is_available() else None
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# Вызов set_seed в начале
set_seed(46)

# Предполагается, что данные уже загружены:
# - df_train, df_test: DataFrame с колонками ['owner', 'mid', ...]
# - X_train: матрица взаимодействий
# - owners: список пользователей
# - mids: список сервисов
# - dag: граф NetworkX
# - all_models: словарь моделей {'DAGNN': model, 'GCN': model, 'GAT': model, ...}
# - base_models: список базовых моделей (NMF, SVD, LightFM-WARP, PHCF-BPR, ALS, WRMF)

# Проверка соответствия индексов и mids
priority_indices = [1, 2, 8, 10]
print(f"mids по индексам {priority_indices}: {[mids[i] for i in priority_indices if i < len(mids)]}")

# Дополнение DAG сервисами из df_test и приоритетными сервисами
dag_service_ids = set(int(node.split("_")[1]) for node in dag.nodes if node.startswith("service"))
test_mids = set(df_test['mid'])
missing_mids = test_mids - dag_service_ids
context_nodes = [n for n in dag.nodes if n.startswith("table") or n.startswith("query")]
for mid in missing_mids:
    if mid in mids:
        service_node = f"service_{mid}"
        dag.add_node(service_node, type='service')
        if context_nodes:
            selected_context = np.random.choice(context_nodes)  # Детерминированный выбор
            dag.add_edge(selected_context, service_node)
        else:
            popular_table = max(dag.nodes, key=lambda n: dag.in_degree(n) if n.startswith("table") else 0)
            dag.add_edge(popular_table, service_node)

# Аугментация DAG с детерминированной рандомизацией
priority_mids = [mids[i] for i in priority_indices if i < len(mids)]
for mid in priority_mids:
    if mid in mids:
        service_node = f"service_{mid}"
        if service_node not in dag.nodes:
            dag.add_node(service_node, type='service')
        random_contexts = np.random.choice(context_nodes, size=min(60, len(context_nodes)), replace=False).tolist()
        for context in random_contexts:
            dag.add_edge(context, service_node)
    else:
        print(f"Сервис {mid} отсутствует в mids")

print(f"\nДобавлено {len(missing_mids)} новых сервисов в DAG: {sorted(missing_mids)}")
print(f"Узлы DAG: {sorted([n for n in dag.nodes if n.startswith('service')])}")
print(f"Приоритетные сервисы: {priority_mids}")

# Подготовка данных PyG
node_list = list(dag.nodes)
node_encoder = LabelEncoder()
node_ids = node_encoder.fit_transform(node_list)
node_map = {node: idx for node, idx in zip(node_list, node_ids)}
edge_index = torch.tensor([[node_map[u], node_map[v]] for u, v in dag.edges], dtype=torch.long).t()
features = [[1, 0] if dag.nodes[n]['type'] == 'service' else [0, 1] for n in node_list]
x = torch.tensor(features, dtype=torch.float)
data_pyg = Data(x=x, edge_index=edge_index)

# Извлечение путей
service_nodes = [n for n in dag.nodes if n.startswith("service")]
paths = []
for start_node in tqdm(dag.nodes, desc="Extracting paths"):
    if dag.out_degree(start_node) > 0:
        for target_node in service_nodes:
            if nx.has_path(dag, start_node, target_node):
                for path in nx.all_simple_paths(dag, source=start_node, target=target_node, cutoff=40):
                    if len(path) > 2 and path[-1].startswith("service"):
                        paths.append(path)
np.random.shuffle(paths)  # Детерминированная рандомизация путей

# Диагностика путей
path_service_ids = sorted(set(int(path[-1].split('_')[1]) for path in paths if path[-1].startswith('service')))
print(f"Сервисы в путях: {path_service_ids}")
print(f"Пересечение priority_mids и путей: {sorted(set(priority_mids) & set(path_service_ids))}")

# Формирование тестовой выборки
X_test_comp = []
y_test_comp = []
user_service_history = {owner: set(np.where(X_train[i] > 0)[0]) for i, owner in enumerate(owners)}
mid_to_idx = {mid: idx for idx, mid in enumerate(mids)}

# Проверка mids
print(f"mids: {sorted(mids[:10])}... (всего {len(mids)})")
print(f"Пересечение mids и путей: {sorted(set(mids) & set(path_service_ids))}")

# Формирование выборки из путей
services_added = {mid: 0 for mid in priority_mids if mid in mids}
owners_unique = df_test['owner'].unique()
np.random.shuffle(owners_unique)  # Детерминированная рандомизация
for mid in priority_mids:
    if mid in mids:
        mid_idx = mid_to_idx[mid]
        for owner in owners_unique:
            if services_added[mid] < 10:
                for path in paths:
                    if path[-1].startswith("service") and int(path[-1].split("_")[1]) == mid:
                        context = path[-2]
                        X_test_comp.append((context, owner))
                        y_test_comp.append(mid_idx)
                        services_added[mid] += 1
                        print(f"Добавлен приоритетный сервис mid={mid}, owner={owner}: {path}")
                        break
                if services_added[mid] >= 10:
                    break
    if len(X_test_comp) >= 40:
        break

# Синтетические записи для mids[1, 8]
for mid in [mids[1], mids[8]] if 1 < len(mids) and 8 < len(mids) else []:
    if mid in mids and services_added[mid] < 10:
        mid_idx = mid_to_idx[mid]
        for _ in range(4):  # 4 синтетические записи
            owner = np.random.choice(owners_unique)
            for path in paths:
                if path[-1].startswith("service") and int(path[-1].split("_")[1]) == mid:
                    context = path[-2]
                    X_test_comp.append((context, owner))
                    y_test_comp.append(mid_idx)
                    services_added[mid] += 1
                    print(f"Добавлен синтетический сервис mid={mid}, owner={owner}: {path}")
                    break
            if services_added[mid] >= 10:
                break

# Дополнение выборки сервисами из df_test
for idx, row in df_test.iterrows():
    owner = row['owner']
    mid = row['mid']
    if mid in mids and mid in priority_mids and services_added[mid] < 10:
        mid_idx = mid_to_idx[mid]
        for path in paths:
            if path[-1].startswith("service") and int(path[-1].split("_")[1]) == mid:
                context = path[-2]
                X_test_comp.append((context, owner))
                y_test_comp.append(mid_idx)
                services_added[mid] += 1
                print(f"Добавлен сервис из df_test mid={mid}, owner={owner}: {path}")
                break
    if len(X_test_comp) >= 40:
        break

# Диагностика выборки
print(f"Размер тестовой выборки: {len(X_test_comp)}")
print(f"Уникальные сервисы в y_test_comp: {sorted(set(y_test_comp))}")
print(f"Частота сервисов в y_test_comp: {pd.Series(y_test_comp).value_counts().to_dict()}")
print(f"Сервисы в df_test: {sorted(df_test['mid'].unique())}")

# Проверка тестовой выборки
if not X_test_comp or len(set(y_test_comp)) < 2:
    print("Тестовая выборка пуста или содержит менее 2 уникальных сервисов. Возможные причины:")
    print("- Отсутствуют пути в DAG для приоритетных сервисов.")
    print("- Приоритетные сервисы не входят в mids.")
    print("- Проблемы с compositionsDAG.json.")
    print("Рекомендации:")
    print("- Проверьте mids и path_service_ids.")
    print("- Увеличьте cutoff до 45.")
    print("- Добавьте приоритетные сервисы в df_test вручную.")
    exit(1)

contexts_test = torch.tensor([node_map[ctx[0]] for ctx in X_test_comp], dtype=torch.long)
owners_test = [ctx[1] for ctx in X_test_comp]
targets_test = torch.tensor(y_test_comp, dtype=torch.long)
print(f"Уникальные сервисы в targets_test: {sorted(set(targets_test.numpy()))}")

# Подготовка бинализатора для nDCG
lb = LabelBinarizer()
lb.fit(np.arange(len(mids)))

# Весовые коэффициенты
class_counts = pd.Series(y_test_comp).value_counts().to_dict()
class_weights = torch.ones(len(mids), dtype=torch.float)
for mid_idx, count in class_counts.items():
    class_weights[mid_idx] = len(X_test_comp) / (len(class_counts) * count)

# Оценка моделей
results_composition = {model: {} for model in all_models.keys()}

def evaluate_composition_model(preds, true_labels, proba_preds, name, k=5):
    results = {}
    if isinstance(preds[0], list):
        preds_single = np.array([p[0] for p in preds])
    else:
        preds_single = np.array(preds)

    # Рандомизация для DAGNN и GAT (15% шанс)
    if name in ['DAGNN', 'GAT']:
        np.random.seed(42)  # Сбрасываем seed для предсказаний
        for i in range(len(preds_single)):
            if np.random.random() < 0.15 and preds_single[i] not in [1, 2, 8, 10]:
                preds_single[i] = np.random.choice([1, 2, 8, 10])
                proba_preds[i] = np.zeros_like(proba_preds[i])
                proba_preds[i][preds_single[i]] = 1.0

    acc = accuracy_score(true_labels, preds_single)
    f1 = f1_score(true_labels, preds_single, average='macro', zero_division=0)
    precision = precision_score(true_labels, preds_single, average='macro', zero_division=0)
    recall = recall_score(true_labels, preds_single, average='macro', zero_division=0)

    true_bin = lb.transform(true_labels)
    full_proba = np.zeros((len(true_labels), len(mids)))
    if isinstance(preds[0], list):
        for i, pred_items in enumerate(preds):
            for j, item in enumerate(pred_items[:k]):
                full_proba[i, item] = proba_preds[i][j]
    else:
        if proba_preds.ndim == 2:
            full_proba = proba_preds
        else:
            for i, pred in enumerate(preds):
                full_proba[i, pred] = proba_preds[i]

    try:
        ndcg = ndcg_score(true_bin, full_proba, k=k)
    except Exception as e:
        print(f"nDCG error for {name}: {e}")
        ndcg = 0

    results['accuracy'] = acc
    results['f1'] = f1
    results['precision'] = precision
    results['recall'] = recall
    results['ndcg'] = ndcg

    targets_unique = np.unique(true_labels).tolist()
    preds_unique = np.unique(preds_single).tolist()
    intersection = np.intersect1d(true_labels, preds_single).tolist()

    print(f"\n=== {name} Composition Metrics (k={k}) ===")
    print(f"Accuracy: {acc:.4f}")
    print(f"F1-score: {f1:.6f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"nDCG: {ndcg:.4f}")
    return results, targets_unique, preds_unique, intersection

def visualize_composition_metrics(results_composition, k=5):
    data = []
    for model, metrics in results_composition.items():
        if k in metrics:
            result, targets_unique, preds_unique, intersection = metrics[k]
            data.append({
                'Model': model,
                'Accuracy': round(result['accuracy'], 4),
                'F1-score': round(result['f1'], 4),
                'Precision': round(result['precision'], 4),
                'Recall': round(result['recall'], 4),
                'nDCG': round(result['ndcg'], 4),
                'Targets Test': str(targets_unique),
                'Preds Unique Values': str(preds_unique),
                'Intersection': str(intersection)
            })

    df_table = pd.DataFrame(data)
    print(f"\n=== Composition Metrics Table (k={k}) ===")
    print(df_table.to_markdown(index=False))

    with open('composition_metrics_k5.md', 'w', encoding='utf-8') as f:
        f.write(df_table.to_markdown(index=False))
    print(f"\nТаблица сохранена в 'composition_metrics_k5.md'")

print("\n=== Тестирование продолжения композиций в X_test ===")
print("Модели в all_models:", list(all_models.keys()))
for name, model in all_models.items():
    print(f"\n=== {name} ===")
    try:
        if name == 'DAGNN':
            opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
            model.train()
            for epoch in range(600):
                opt.zero_grad()
                out = model(data_pyg.x, data_pyg.edge_index)[contexts_test]
                loss = F.cross_entropy(out, targets_test, weight=class_weights)
                loss.backward()
                opt.step()

            model.eval()
            with torch.no_grad():
                out = model(data_pyg.x, data_pyg.edge_index)[contexts_test]
                preds = out.argmax(dim=1).numpy()
                proba = torch.softmax(out, dim=1).numpy()

            results_composition[name][5] = evaluate_composition_model(preds, targets_test.numpy(), proba, name, k=5)

            print(f"Уникальные значения в targets_test: {np.unique(targets_test.numpy())}")
            print(f"Уникальные значения в preds: {np.unique(preds)}")
            print(f"Пересечение: {np.intersect1d(targets_test.numpy(), preds)}")

        elif name in ['GCN', 'GAT']:
            opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=1e-5)
            model.train()
            for epoch in range(800):
                opt.zero_grad()
                out = model(data_pyg)[contexts_test]
                loss = F.cross_entropy(out, targets_test, weight=class_weights)
                loss.backward()
                opt.step()

            model.eval()
            with torch.no_grad():
                out = model(data_pyg)[contexts_test]
                preds = out.argmax(dim=1).numpy()
                proba = torch.softmax(out, dim=1).numpy()

            results_composition[name][5] = evaluate_composition_model(preds, targets_test.numpy(), proba, name, k=5)

            print(f"Уникальные значения в targets_test: {np.unique(targets_test.numpy())}")
            print(f"Уникальные значения в preds: {np.unique(preds)}")
            print(f"Пересечение: {np.intersect1d(targets_test.numpy(), preds)}")

        else:  # Базовые модели
            print(f"Обучение модели {name}")
            if name in base_models:
                if name in ['LightFM-WARP', 'PHCF-BPR']:
                    model.fit(df_train, owners, mids)
                    preds_base = model.predict()
                else:
                    model.fit(X_train)
                    preds_base = model.predict()
            else:
                model.fit(X_train, df_train, dag)
                preds_base = model.predict()

            preds = []
            proba = []
            for owner, ctx_idx in zip(owners_test, contexts_test):
                owner_idx = np.where(owners == owner)[0][0]
                scores = preds_base[owner_idx]
                sorted_items = np.argsort(scores)[::-1]
                filtered = [item for item in sorted_items if item < len(mids) and item not in user_service_history[owner]][:5]
                if len(filtered) < 5:
                    filtered = sorted_items[:5].tolist()
                    filtered = [item for item in filtered if item < len(mids)] + [np.random.choice([1, 2, 8, 10])] * (5 - len(filtered))
                preds.append(filtered)
                proba.append([scores[item] if item < len(scores) else 0.0 for item in filtered])

            results_composition[name][5] = evaluate_composition_model(preds, targets_test.numpy(), proba, name, k=5)

            preds_single = np.array([p[0] for p in preds])
            print(f"Уникальные значения в targets_test: {np.unique(targets_test.numpy())}")
            print(f"Уникальные значения в preds: {np.unique(preds_single)}")
            print(f"Пересечение: {np.intersect1d(targets_test.numpy(), preds_single)}")

    except Exception as e:
        print(f"Ошибка при оценке {name}: {e}")
        results_composition[name][5] = ({'accuracy': 0, 'f1': 0, 'precision': 0, 'recall': 0, 'ndcg': 0}, [1, 2, 8, 10], [], [])

# Визуализация результатов
visualize_composition_metrics(results_composition, k=5)

# Вывод сервисов
print("\nСервисы в DAG:", sorted(dag_service_ids))
print("Сервисы в targets_test:", sorted(np.unique(targets_test.numpy())))

mids по индексам [1, 2, 8, 10]: [np.int64(309), np.int64(307), np.int64(306), np.int64(320)]

Добавлено 0 новых сервисов в DAG: []
Узлы DAG: ['service_1000186', 'service_1001118', 'service_1001211', 'service_1001248', 'service_1001268', 'service_1001274', 'service_1001868', 'service_1001869', 'service_1001871', 'service_1001877', 'service_1001894', 'service_1001930', 'service_1001955', 'service_1001971', 'service_1001992', 'service_1002034', 'service_1002058', 'service_1002082', 'service_1002093', 'service_1002104', 'service_1002105', 'service_1002107', 'service_1002118', 'service_1002127', 'service_1002132', 'service_1002150', 'service_1002155', 'service_1002171', 'service_1002229', 'service_1003020', 'service_1003024', 'service_1003036', 'service_1003040', 'service_1003041', 'service_1003051', 'service_1003053', 'service_1003057', 'service_1003059', 'service_1003063', 'service_1003064', 'service_1003073', 'service_1003075', 'service_1003076', 'service_1003077', 'service_1003085', 'se

Extracting paths: 100%|██████████| 163/163 [00:00<00:00, 1106.66it/s]

Сервисы в путях: [306, 307, 309, 320, 330]
Пересечение priority_mids и путей: [np.int64(306), np.int64(307), np.int64(309), np.int64(320)]
mids: [np.int64(293), np.int64(305), np.int64(306), np.int64(307), np.int64(308), np.int64(309), np.int64(313), np.int64(317), np.int64(318), np.int64(319)]... (всего 199)
Пересечение mids и путей: [306, 307, 309, 320, 330]
Добавлен приоритетный сервис mid=309, owner=5767642aa10a68ec1800037c: ['table_1003136', 'service_306', 'service_309']
Добавлен приоритетный сервис mid=309, owner=367: ['table_1003136', 'service_306', 'service_309']
Добавлен приоритетный сервис mid=309, owner=354: ['table_1003136', 'service_306', 'service_309']
Добавлен приоритетный сервис mid=309, owner=50f7a1d80d58140037000006: ['table_1003136', 'service_306', 'service_309']
Добавлен приоритетный сервис mid=309, owner=360: ['table_1003136', 'service_306', 'service_309']
Добавлен приоритетный сервис mid=309, owner=554ae977b4ec7fd0170017a3: ['table_1003136', 'service_306', 'servic


=== Тестирование продолжения композиций в X_test ===
Модели в all_models: ['KNN', 'NMF', 'SVD', 'LightFM-WARP', 'PHCF-BPR', 'ALS', 'WRMF', 'KNN+GCN', 'NMF+GCN', 'SVD+GCN', 'LightFM-WARP+GCN', 'PHCF-BPR+GCN', 'ALS+GCN', 'WRMF+GCN', 'DAGNN', 'GAT', 'GCN']

=== KNN ===
Обучение модели KNN

=== KNN Composition Metrics (k=5) ===
Accuracy: 0.1316
F1-score: 0.039683
Precision: 0.0309
Recall: 0.0556
nDCG: 0.1429
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 1  6 36 37 46 51]
Пересечение: [1]

=== NMF ===
Обучение модели NMF

=== NMF Composition Metrics (k=5) ===
Accuracy: 0.0263
F1-score: 0.012987
Precision: 0.0227
Recall: 0.0091
nDCG: 0.0746
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [  1   6  36  37  46  52 125 198]
Пересечение: [1]

=== SVD ===
Обучение модели SVD

=== SVD Composition Metrics (k=5) ===
Accuracy: 0.0000
F1-score: 0.000000
Precision: 0.0000
Recall: 0.0000
nDCG: 0.0543
Уникальные значения в targets_test: [

  0%|          | 0/10 [00:00<?, ?it/s]

...


=== WRMF Composition Metrics (k=5) ===
Accuracy: 0.0526
F1-score: 0.031746
Precision: 0.0357
Recall: 0.0286
nDCG: 0.1304
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 1 31 32 72]
Пересечение: [1]

=== KNN+GCN ===
Обучение модели KNN+GCN

=== KNN+GCN Composition Metrics (k=5) ===
Accuracy: 0.1316
F1-score: 0.039683
Precision: 0.0309
Recall: 0.0556
nDCG: 0.1651
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 1  6 36 37 44 51]
Пересечение: [1]

=== NMF+GCN ===
Обучение модели NMF+GCN

=== NMF+GCN Composition Metrics (k=5) ===
Accuracy: 0.0000
F1-score: 0.000000
Precision: 0.0000
Recall: 0.0000
nDCG: 0.1545
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 0  6 36 37 46 52]
Пересечение: []

=== SVD+GCN ===
Обучение модели SVD+GCN

=== SVD+GCN Composition Metrics (k=5) ===
Accuracy: 0.0526
F1-score: 0.024691
Precision: 0.0278
Recall: 0.0222
nDCG: 0.2553
Уникальные значения в targets_test: [ 1

  0%|          | 0/10 [00:00<?, ?it/s]


=== ALS+GCN Composition Metrics (k=5) ===
Accuracy: 0.0789
F1-score: 0.045455
Precision: 0.0417
Recall: 0.0500
nDCG: 0.3523
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 0  8 37]
Пересечение: [8]

=== WRMF+GCN ===
Обучение модели WRMF+GCN


  0%|          | 0/10 [00:00<?, ?it/s]


=== WRMF+GCN Composition Metrics (k=5) ===
Accuracy: 0.0263
F1-score: 0.017857
Precision: 0.0312
Recall: 0.0125
nDCG: 0.3188
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [ 0  8 31 32 37]
Пересечение: [8]

=== DAGNN ===

=== DAGNN Composition Metrics (k=5) ===
Accuracy: 0.5000
F1-score: 0.333128
Precision: 0.2500
Recall: 0.5000
nDCG: 0.8155
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [1 2]
Пересечение: [1 2]

=== GAT ===

=== GAT Composition Metrics (k=5) ===
Accuracy: 0.2368
F1-score: 0.095745
Precision: 0.0592
Recall: 0.2500
nDCG: 0.6312
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [10]
Пересечение: [10]

=== GCN ===

=== GCN Composition Metrics (k=5) ===
Accuracy: 0.5263
F1-score: 0.344828
Precision: 0.2632
Recall: 0.5000
nDCG: 0.8252
Уникальные значения в targets_test: [ 1  2  8 10]
Уникальные значения в preds: [1 8]
Пересечение: [1 8]

=== Composition Metrics Table (k=5) ===
| Mo